In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('data-preprocessed.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
dataset.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,M
1,842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,M
2,84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,M
3,84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,M
4,84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,M


In [3]:
# Import label encoder
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
dataset['diagnosis']= label_encoder.fit_transform(dataset['diagnosis'])
 
dataset['diagnosis'].unique()
dataset.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


Training and testing set



In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#X_train = X_train.reshape(-1,1)
X_train = sc.fit_transform(X_train)#.reshape((len(X_train), 1)))
X_test = sc.transform(X_test)#.reshape(-1,1))

In [5]:
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif, k=6)
selector.fit(X, y)

SelectKBest(k=6)

In [6]:
cols = selector.get_support(indices=True)
cols

array([ 3,  8, 21, 23, 24, 28])

In [7]:
dataset.drop(columns=['id','texture_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','texture_worst','smoothness_worst','compactness_worst','concavity_worst','symmetry_worst','fractal_dimension_worst'])

,radius_mean,perimeter_mean,concave points_mean,radius_worst,perimeter_worst,area_worst,concave points_worst,diagnosis
0,17.99,122.80,0.14710,25.380,184.60,2019.0,0.2654,1
1,20.57,132.90,0.07017,24.990,158.80,1956.0,0.1860,1
2,19.69,130.00,0.12790,23.570,152.50,1709.0,0.2430,1
3,11.42,77.58,0.10520,14.910,98.87,567.7,0.2575,1
4,20.29,135.10,0.10430,22.540,152.20,1575.0,0.1625,1
...,...,...,...,...,...,...,...,...
564,21.56,142.00,0.13890,25.450,166.10,2027.0,0.2216,1
565,20.13,131.20,0.09791,23.690,155.00,1731.0,0.1628,1
566,16.60,108.30,0.05302,18.980,126.70,1124.0,0.1418,1
567,20.60,140.10,0.15200,25.740,184.60,1821.0,0.2650,1


In [8]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [ ]:
#With Hyper Parameters Tuning
#2-2,Randomforest
#importing modules
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
#making the instance
model=RandomForestClassifier()
#hyper parameters set
params = {'criterion':['gini','entropy'],
          'n_estimators':[n for n in range(1,150,15)],
          'min_samples_leaf':[1,2,3],
          'min_samples_split':[3,4,5,6,7], 
          'random_state':[123],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
#learning
model1.fit(train_X,train_y)
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)
#Prediction
rf_prediction=model1.predict(test_X)
train_predict=model1.predict(train_X)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Training set Accuracy using hyperparameter",metrics.accuracy_score(train_predict,train_y))
print("Testing set Accuracy:",metrics.accuracy_score(rf_prediction,test_y))
#evaluation(Confusion Metrix)
# print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,test_y))


Best Hyper Parameters:
 {'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 46, 'n_jobs': -1, 'random_state': 123}
Training set Accuracy using hyperparameter 0.9949748743718593
Testing set Accuracy: 0.9649122807017544


In [10]:
#With Hyper Parameters Tuning
#AdaBoostClassifier
#importing modules
# from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
#making the instance
abc=AdaBoostClassifier()
#learning
abc.fit(train_X, train_y)
#Preadiction
abc_prediction=abc.predict(test_X)
abc_train_predict=abc.predict(train_X)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Training set Accuracy:",metrics.accuracy_score(abc_train_predict,train_y))
print("Testing set Accuracy:",metrics.accuracy_score(abc_prediction,test_y))

Training set Accuracy: 1.0
Testing set Accuracy: 0.9707602339181286


In [11]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


modelxg = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
modelxg.fit(train_X,train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

y_pred = modelxg.predict(test_X)
y_pred

print("Accuracy:-")
accuracy = accuracy_score(test_y, y_pred)
accuracy

Accuracy:-


0.9766081871345029